### LABELED DATA

In [1]:
from utils.models import DataItem
lines =open('/home/ec2-user/SageMaker/mariano/notebooks/05. High Recall V2/labeled_data.csv','r').read().splitlines()[1:]
lines = [line.split(';') for line in lines ]
lines[:3]
labeled_data = [DataItem(id_) for id_,_ in lines]

idx = 0
for id_,label in lines:
        item = labeled_data[idx]
        idx+=1
        assert item.id_ == id_
        if label == 'R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'


lines = open('./auxiliary_notebooks/in_canada.csv', 'r').read().splitlines()
irrelevants = [DataItem(line.split(';')[0]) for line in lines if line.split(';')[1]=='False']
for item in irrelevants:
    item.set_irrelevant()
labeled_data = labeled_data + irrelevants
print(f'{len(labeled_data):,}')

114,492


In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from utils.general import info
import pandas as pd
params = {'C':[0.5, 1, 5, 7, 15, 50]}
GS = GridSearchCV(
                  SVC(kernel='rbf', probability=True),
                  params
                  ,scoring=['f1','accuracy','precision', 'recall'], 
                  cv=5,
                  verbose=4, 
                  refit='f1')    

X = DataItem.get_X(labeled_data, type_=DataItem.TYPE_GLOVE300)
y = DataItem.get_y(labeled_data)
rta = GS.fit(X,y)
df = (pd.DataFrame(rta.cv_results_)[["params","mean_test_accuracy", "mean_test_precision","mean_test_recall","mean_test_f1"]])
df.columns=['params','mean accuracy', 'mean precision','mean recall','mean f1',]
print(df) 
info(f'Best Params: {rta.best_params_}')
clf_g300 = GS.best_estimator_

del(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END C=0.5; accuracy: (test=0.988) f1: (test=0.438) precision: (test=0.292) recall: (test=0.880) total time= 2.6min
[CV 2/5] END C=0.5; accuracy: (test=0.997) f1: (test=0.659) precision: (test=1.000) recall: (test=0.491) total time= 3.2min
[CV 3/5] END C=0.5; accuracy: (test=0.998) f1: (test=0.682) precision: (test=1.000) recall: (test=0.517) total time= 3.7min
[CV 4/5] END C=0.5; accuracy: (test=0.997) f1: (test=0.568) precision: (test=1.000) recall: (test=0.397) total time= 5.0min
[CV 5/5] END C=0.5; accuracy: (test=0.997) f1: (test=0.640) precision: (test=0.982) recall: (test=0.474) total time= 5.1min
[CV 1/5] END C=1; accuracy: (test=0.987) f1: (test=0.420) precision: (test=0.270) recall: (test=0.949) total time= 3.1min
[CV 2/5] END C=1; accuracy: (test=0.998) f1: (test=0.766) precision: (test=1.000) recall: (test=0.621) total time= 3.0min
[CV 3/5] END C=1; accuracy: (test=0.998) f1: (test=0.814) precision: (test=0

NameError: name 'pd' is not defined

In [5]:
from utils.general import info
import pandas as pd
df = (pd.DataFrame(rta.cv_results_)[["params","mean_test_accuracy", "mean_test_precision","mean_test_recall","mean_test_f1"]])
df.columns=['params','mean accuracy', 'mean precision','mean recall','mean f1',]
print(df)
info(f'Best Params: {rta.best_params_}')
clf_g300 = GS.best_estimator_

del(X,y)

       params  mean accuracy  mean precision  mean recall   mean f1
0  {'C': 0.5}       0.995537        0.854786     0.551930  0.597302
1    {'C': 1}       0.995816        0.846598     0.696640  0.701256
2    {'C': 5}       0.995764        0.842157     0.779340  0.749679
3    {'C': 7}       0.995659        0.842795     0.784512  0.751867
4   {'C': 15}       0.995554        0.834871     0.808635  0.761776
5   {'C': 50}       0.995074        0.803696     0.824167  0.756359
2022-03-25 16:45:04.694836 [ INFO  ] Best Params: {'C': 15}


In [6]:
clf_g300

SVC(C=15, probability=True)

In [11]:
import os
prediction_files = os.listdir('../04. Model of DP/predictions/')
suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]
print(suggestions[0])

DataItem(id=1323603426, source=GM1, label=U)


In [27]:
import os
import pickle
import numpy as np
prediction_files = os.listdir('../04. Model of DP/predictions/')
suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]

X = DataItem.get_X(suggestions, type_=DataItem.TYPE_GLOVE300)

yhat = clf_g300.predict_proba(X)

idx=0
for item in suggestions:
    prediction_file = '../04. Model of DP/predictions/'+item.id_+'_v3.p'
    assert os.path.isfile(prediction_file)
    
    y = pickle.load(open(prediction_file,'rb'))
    
    assert len(y)==3 and np.sum(y)==0 and y.shape==(3,)
    y[0] = yhat[idx,1]
#     print(y)
    pickle.dump(y, open(prediction_file,'wb'))
    idx+=1


In [24]:
yhat[0:5,:]

array([[0.93740742, 0.06259258],
       [0.5       , 0.5       ],
       [0.56556452, 0.43443548],
       [0.58218613, 0.41781387],
       [0.5       , 0.5       ]])